# Algoritmo de BackPropagation multiclase

Este codigo se utilizará para entrenar la red neuronal de clasificacion de Nuestras Caras



### Código en Python del Algoritmo de BackPropagation

La clase del dataset debe ser categorica
Existe una clase en Python que resuelve el problema

In [ ]:
# conexion al Google Drive
from google.colab import drive
drive.mount('/content/.drive')
!mkdir -p "/content/.drive/My Drive/DMA"
!mkdir -p "/content/buckets"
!ln -s "/content/.drive/My Drive/DMA" /content/buckets/b1

In [ ]:
# instalo  itables solo si no esta instalado
!pip show itables >/dev/null || pip install itables

In [ ]:
import polars as pl
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import time
import os
import pickle
from functools import reduce
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)


In [ ]:
# definicion de la clase de graficos

class perceptron_plot:
    """plotting first hidden layer class"""
    def __init__(self, X, Y, delay) -> None:
        self.X = X
        self.Y = Y
        self.delay = delay
        x1_min = np.min(X[:,0])
        x2_min = np.min(X[:,1])
        x1_max = np.max(X[:,0])
        x2_max = np.max(X[:,1])
        self.x1_min = x1_min - 0.1*(x1_max - x1_min)
        self.x1_max = x1_max + 0.1*(x1_max - x1_min)
        self.x2_min = x2_min - 0.1*(x2_max - x2_min)
        self.x2_max = x2_max + 0.1*(x2_max - x2_min)
        self.fig = plt.figure(figsize = (10,8))
        self.ax = self.fig.subplots()
        self.ax.set_xlim(self.x1_min, self.x1_max, auto=False)
        self.ax.set_ylim(self.x2_min, self.x2_max, auto=False)

    def graficarVarias(self, W, x0, epoch, error) -> None:
        display.clear_output(wait =True)
        plt.cla()
        #self.ax = self.fig.subplots()

        self.ax.set_xlim(self.x1_min, self.x1_max)
        self.ax.set_ylim(self.x2_min, self.x2_max)
        plt.title( 'epoch ' + str(epoch) + '  reg ' + "{0:.2E}".format(error))
        # ploteo puntos
        num_classes = len(np.unique(self.Y))
        # mycolors = plt.cm.get_cmap('tab10', num_classes)
        unique_labels = np.unique(self.Y)
        label_mapping = {label: i for i, label in enumerate(unique_labels)}
        numerical_labels = np.array([label_mapping[label] for label in self.Y])
        # scatter = self.ax.scatter(self.X[:,0], self.X[:,1], c=self.Y, s=20)
        scatter = self.ax.scatter(self.X[:,0], self.X[:,1], c=numerical_labels, s=20, cmap=plt.get_cmap('viridis', num_classes))
        # self.ax.plot(self.X[:,0], self.X[:,1], 'o', c=vcolores,  markersize=2)


        # dibujo las rectas
        for i in range(len(x0)):
            #vx2_min = -(W[0,i]*self.x1_min + x0[i])/W[1,i]
            #vx2_max = -(W[0,i]*self.x1_max + x0[i])/W[1,i]
            vx2_min = -(W[i,0]*self.x1_min + x0[i])/W[i,1]
            vx2_max = -(W[i,0]*self.x1_max + x0[i])/W[i,1]

            self.ax.plot([self.x1_min, self.x1_max],
                         [vx2_min, vx2_max],
                         linewidth = 2,
                         color = 'red',
                         alpha = 0.5)

        display.display(plt.gcf())
        #plt.cla()
        time.sleep(self.delay)


In [ ]:
# definicion de las funciones de activacion
#  y sus derivadas
#  ahora agregando las versiones VECTORIZADAS

def func_eval(fname, x):
    match fname:
        case "purelin":
            y = x
        case "logsig":
            y = 1.0 / ( 1.0 + math.exp(-x) )
        case "tansig":
            y = 2.0 / ( 1.0 + math.exp(-2.0*x) ) - 1.0
    return y

# version vectorizada de func_eval
func_eval_vec = np.vectorize(func_eval)


def deriv_eval(fname, y):  #atencion que y es la entrada y=f( x )
    match fname:
        case "purelin":
            d = 1.0
        case "logsig":
            d = y*(1.0-y)
        case "tansig":
            d = 1.0 - y*y
    return d


# version vectorizada de deriv_eval
deriv_eval_vec = np.vectorize(deriv_eval)

In [ ]:
# prompt: add to func_eval and deriv_eval the softmax function

import numpy as np

def func_eval(fname, x):
    match fname:
        case "purelin":
            y = x
        case "logsig":
            y = 1.0 / ( 1.0 + np.exp(-x) )
        case "tansig":
            y = 2.0 / ( 1.0 + np.exp(-2.0*x) ) - 1.0
        case "softmax":  # Added softmax function
            exp_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
            y = exp_x / np.sum(exp_x, axis=0)
    return y

# Vectorized version remains the same, as numpy handles vectorization for the new cases
func_eval_vec = np.vectorize(func_eval)


def deriv_eval(fname, y):
    match fname:
        case "purelin":
            d = 1.0
        case "logsig":
            d = y * (1.0 - y)
        case "tansig":
            d = 1.0 - y * y
        case "softmax": # Added softmax derivative (Note: this is the Jacobian matrix)
            # Derivative of softmax is a matrix. This returns the diagonal entries
            # for the case where you need the derivative of a single output element with
            # respect to its input.  For full Jacobian, refer to the documentation.
            d = np.diagflat(y) - np.outer(y, y)
    return d

# Vectorized version remains the same, as numpy handles vectorization for the new cases.
deriv_eval_vec = np.vectorize(deriv_eval)


### Clase  multiperceptron
entrenar, predecir

In [ ]:
# definicion de la clase de multiperceptron

class multiperceptron(object):
    """Multiperceptron class"""

    # inicializacion de los pesos de todas las capas
    def _red_init(self, semilla) -> None:
        niveles = self.red['arq']['layers_qty']

        np.random.seed(semilla)
        for i in range(niveles):
           nivel = dict()
           nivel['id'] = i
           nivel['last'] = (i==(niveles-1))
           nivel['size'] = self.red["arq"]["layers_size"][i]
           nivel['func'] = self.red["arq"]["layers_func"][i]

           if( i==0 ):
              entrada_size = self.red['arq']['input_size']
           else:
              entrada_size =  self.red['arq']['layers_size'][i-1]

           salida_size =  nivel['size']

           # los pesos, inicializados random
           nivel['W'] = np.random.uniform(-0.5, 0.5, [salida_size, entrada_size])
           nivel['w0'] = np.random.uniform(-0.5, 0.5, [salida_size, 1])

           # los momentos, inicializados en CERO
           nivel['W_m'] = np.zeros([salida_size, entrada_size])
           nivel['w0_m'] = np.zeros([salida_size, 1])

           self.red['layer'].append(nivel)

    # constructor generico
    def __init__(self) -> None:
        self.data = dict()
        self.red = dict()
        self.carpeta = ""


    # inicializacion full
    def inicializar(self, df, campos, clase, hidden_layers_sizes, layers_func,
                 semilla, carpeta) -> None:

        # genero self.data
        self.data['X'] = np.array( df.select(campos))
        X_mean = self.data['X'].mean(axis=0)
        X_sd = self.data['X'].std(axis=0)
        self.data['X'] = (self.data['X'] - X_mean)/X_sd

        #  Ylabel en  numpy
        label =df.select(clase)
        self.data['Ylabel'] = np.array(label).reshape(len(label))

        # one-hot-encoding de Y
        col_originales = df.columns
        self.data['Y'] = np.array( df.to_dummies(clase).drop(col_originales, strict=False) )

        col_dummies = sorted( list( set(df.to_dummies(clase).columns) -  set(col_originales)))
        clases_originales = reduce(lambda acc, x: acc + [x[(len(clase)+1):]], col_dummies, [])

        tamanos = hidden_layers_sizes
        tamanos.append(self.data['Y'].shape[1])

        arquitectura = {
             'input_size' : self.data['X'].shape[1],
             'input_mean' : X_mean,
             'input_sd' :  X_sd,
             'output_values' : clases_originales,
             'layers_qty' : len(hidden_layers_sizes), # incluye la capa de salida, pero no la de entrada
             'layers_size' : tamanos ,
             'layers_func' : layers_func,
        }

        self.red['arq'] = arquitectura


        # inicializo  work
        self.red['work'] = dict()
        self.red['work']['epoch'] = 0
        self.red['work']['MSE'] = float('inf')
        self.red['work']['train_error_rate'] = float('inf')

        self.red['layer'] = list()
        self._red_init(semilla)

        # grabo el entorno
        self.carpeta = carpeta
        os.makedirs(self.carpeta, exist_ok=True)
        with open(self.carpeta+"/data.pkl", 'wb') as f:
            pickle.dump(self.data, f)

        with open(self.carpeta+"/red.pkl", 'wb') as f:
            pickle.dump(self.red, f)





    # Algoritmo Backpropagation
    def  entrenar(self, epoch_limit, MSE_umbral,
               learning_rate, lr_momento, save_frequency,
               retomar=True) -> None:

        # si debo retomar
        if( retomar):
            with open(self.carpeta+"/data.pkl", 'rb') as f:
              self.data = pickle.load(f)

            with open(self.carpeta+"/red.pkl", 'rb') as f:
              self.red = pickle.load(f)


        # inicializaciones del bucle principal del backpropagation
        epoch = self.red['work']['epoch']
        MSE = self.red['work']['MSE']

        # inicializacion del grafico
        grafico = perceptron_plot(X=self.data['X'], Y=self.data['Ylabel'], delay=0.1)

        # continuo mientras error cuadratico medio muy grande  y NO llegué al límite de epochs
        Xfilas = self.data['X'].shape[0]
        niveles = self.red["arq"]["layers_qty"]

        while ( MSE > MSE_umbral) and (epoch < epoch_limit) :
          epoch += 1


          # recorro siempre TODOS los registros de entrada
          for fila in range(Xfilas):
             # fila es el registro actual
             x = self.data['X'][fila:fila+1,:]
             clase = self.data['Y'][fila:fila+1,:]

             # propagar el x hacia adelante, FORWARD
             entrada = x.T  # la entrada a la red

             # etapa forward
             # recorro hacia adelante, nivel a nivel
             vsalida =  [0] *(niveles) # salida de cada nivel de la red

             for i in range(niveles):
               estimulos = self.red['layer'][i]['W'] @ entrada + self.red['layer'][i]['w0']
               vsalida[i] =  func_eval_vec(self.red['layer'][i]['func'], estimulos)
               entrada = vsalida[i]  # para la proxima vuelta


             # etapa backward
             # calculo los errores en la capa hidden y la capa output
             verror =  [0] *(niveles+1) # inicializo dummy
             verror[niveles] = clase.T - vsalida[niveles-1]

             i = niveles-1
             verror[i] = verror[i+1] * deriv_eval_vec(self.red['layer'][i]['func'], vsalida[i])

             for i in reversed(range(niveles-1)):
               verror[i] = deriv_eval_vec(self.red['layer'][i]['func'], vsalida[i])*(self.red['layer'][i+1]['W'].T @ verror[i+1])

             # ya tengo los errores que comete cada capa
             # corregir matrices de pesos, voy hacia atras
             # backpropagation
             entrada = x.T
             for i in range(niveles):
               self.red['layer'][i]['W_m'] = learning_rate *(verror[i] @ entrada.T) + lr_momento *self.red['layer'][i]['W_m']
               self.red['layer'][i]['w0_m'] = learning_rate * verror[i] + lr_momento * self.red['layer'][i]['w0_m']

               self.red['layer'][i]['W']  =  self.red['layer'][i]['W'] + self.red['layer'][i]['W_m']
               self.red['layer'][i]['w0'] =  self.red['layer'][i]['w0'] + self.red['layer'][i]['w0_m']
               entrada = vsalida[i]  # para la proxima vuelta



          # ya recalcule las matrices de pesos
          # ahora avanzo la red, feed-forward
          # para calcular el red(X) = Y
          entrada = self.data['X'].T
          for i in range(niveles):
            estimulos = self.red['layer'][i]['W'] @ entrada + self.red['layer'][i]['w0']
            salida =  func_eval_vec(self.red['layer'][i]['func'], estimulos)
            entrada = salida  # para la proxima vuelta

          # calculo el error cuadratico medio TODOS los X del dataset
          MSE= np.mean( (self.data['Y'].T - salida)**2 )

          # Grafico las rectas SOLAMENTE de la Primera Hidden Layer
          # tengo que hacer w0.T[0]  para que pase el vector limpio
          if( epoch % save_frequency == 0 ) or ( MSE <= MSE_umbral) or (epoch >= epoch_limit) :
              # grafico
              W = self.red['layer'][0]['W']
              w0 = self.red['layer'][0]['w0']
              grafico.graficarVarias(W, w0.T[0], epoch, MSE)

              # almaceno en work
              self.red['work']['epoch'] = epoch
              self.red['work']['MSE'] = MSE
              prediccion = np.argmax( salida.T, axis=1)
              # prediccion
              out = np.array(self.red["arq"]['output_values'])
              error_rate = np.mean( self.data['Ylabel'] != out[prediccion])
              self.red["work"]['train_error_rate'] = error_rate # error_rate != error cuadratico medio

              # grabo a un archivo la red neuronal  entrenada por donde esté
              #   solo la red, NO los datos
              with open(carpeta+"/red.pkl", 'wb') as f:
                 pickle.dump(self.red, f)

        return (epoch, MSE, self.red['work']['train_error_rate'] )


    # predigo a partir de modelo recien entrenado
    def  predecir(self, df_new, campos, clase) -> None:
        niveles = self.red['arq']['layers_qty']

        # etapa forward
        # recorro hacia adelante, nivel a nivel
        X_new =  np.array( df_new.select(campos))


        # estandarizo manualmente
        #  con las medias y desvios que almacene durante el entrenamiento
        X_new = (X_new - self.red['arq']['input_mean'])/self.red['arq']['input_sd']

        # grafico los datos nuevos
        Ylabel_new =df_new.select(clase)
        Ylabel_new = np.array(Ylabel_new).reshape(len(Ylabel_new))
        grafico = perceptron_plot(X=X_new, Y=Ylabel_new, delay=0.1)
        W = self.red['layer'][0]['W']
        w0 = self.red['layer'][0]['w0']
        grafico.graficarVarias(W, w0.T[0], epoch, MSE)

        # la entrada a la red,  el X que es TODO  x_new
        entrada = X_new.T  # traspongo, necesito vectores columna

        for i in range(niveles):
          estimulos = self.red['layer'][i]['W'] @ entrada + self.red['layer'][i]['w0']
          salida =  func_eval_vec(self.red['layer'][i]['func'], estimulos)
          entrada = salida  # para la proxima vuelta

        # me quedo con la neurona de la ultima capa que se activio con mayor intensidad
        pred_idx = np.argmax( salida.T, axis=1)
        pred_raw = np.max( salida.T, axis=1)

        # calculo error_rate
        out = np.array(self.red['arq']['output_values'])
        error_rate = np.mean( np.array(df_new.select("y") != out[pred_idx]))

        return (out[pred_idx], pred_raw, error_rate)


    # cargo un modelo ya entrenado, grabado en carpeta
    def cargar_modelo(self, carpeta) -> None:
        self.carpeta = carpeta

        with open(self.carpeta+"/red.pkl", 'rb') as f:
          self.red = pickle.load(f)

        return (self.red['work']['epoch'],
                self.red['work']['MSE'],
                self.red['work']['train_error_rate'] )


## 1 Lectura del Dataset

En este humilde y restringida version, la clase del dataset debe ser categorica, no es capaz de trabajar con clases continuas.
La clase categorica puede ser  n-aria

In [ ]:
!wget 'https://raw.githubusercontent.com/josekeh/dma_g2/main/input/caras50.pkl'

In [ ]:
import pickle

with open("caras50.pkl", "rb") as file:
    caras = pickle.load(file)

### 1.1 Particion  training/testing




In [ ]:
import random
from collections import Counter

# Algoritmo para chequear la distribución de nombres
def check_separar(training_nombres):
  # Contar la frecuencia de cada palabra
  contador = Counter(training_nombres)
  total_palabras = sum(contador.values())
  cantidad_diferentes = len(contador)  # Número de palabras únicas

  # Convertir a porcentaje
  frecuencias = {palabra: (conteo / total_palabras) * 100 for palabra, conteo in contador.items()}

  umbral = 100 / cantidad_diferentes

  # Mostrar el gráfico si no es muy dispar
  if (min(frecuencias.values()) < umbral*.75 or max(frecuencias.values()) > umbral*1.25):
    return 0
  else:
    # Graficar
    plt.figure(figsize=(12, 6))
    plt.bar(frecuencias.keys(), frecuencias.values(), color='skyblue')

    # Agregar línea horizontal en 100% / cantidad de palabras diferentes
    plt.axhline(y=umbral, color='red', linestyle='--', label=f"Ideal: {umbral:.2f}%")

    # Etiquetas
    plt.xlabel("Persona")
    plt.ylabel("Frecuencia (%)")
    plt.title("Distribución del training set según las personas")
    plt.ylim(0, max(frecuencias.values()) + 5)  # Ajuste del eje Y
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()

# Algoritmo para separar training y testing suponiendo 2 listas
def separar_aleatorio(lista_caras, lista_nombres, porcentaje_training):
    if (len(lista_caras) != len(lista_nombres)):
      print("Error las listas deben tener la misma longitud")
    else:
      chequeo = 0

      while (chequeo == 0):
        training_caras = []
        training_nombres = []
        testing_caras = []
        testing_nombres = []

        for i in range(len(lista_caras)):
          aleatorio = random.random()
          if aleatorio <= porcentaje_training:
            training_caras.append(lista_caras[i])
            training_nombres.append(lista_nombres[i])
          else:
            testing_caras.append(lista_caras[i])
            testing_nombres.append(lista_nombres[i])
        chequeo = check_separar(training_nombres)

      return training_caras, training_nombres, testing_caras, testing_nombres



In [ ]:
faces = [sublista[0] for sublista in caras]
names = [sublista[1] for sublista in caras]

In [ ]:
random.seed(102191)
train_caras, train_nombres, test_caras, test_nombres = separar_aleatorio(faces, names, 0.7)

### Para la corrida final sin testing set

In [ ]:
faces = [sublista[0] for sublista in caras]
names = [sublista[1] for sublista in caras]

In [ ]:
test_caras = []
test_nombres = []
train_caras = []
train_nombres = []
for i in range(len(faces)):
  if i in []:
    test_caras.append(faces[i])
    test_nombres.append(names[i])
  else:
    train_caras.append(faces[i])
    train_nombres.append(names[i])

In [ ]:
for i in range(len(test_caras)):
  print(test_nombres[i])
  plt.figure(figsize=(1, 1))
  plt.imshow(test_caras[i].reshape(50, 50), cmap='gray')
  plt.axis('off')
  plt.show()


### ISOMAP

Aclaración: En caso de estar trabajando con training y testing hay que "descomentar" el código comentado sobre el testing set

In [ ]:
import numpy as np
from sklearn.manifold import Isomap
from sklearn.datasets import make_swiss_roll
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
mejor_n_components = 35
mejor_n_neighbors = 30

In [ ]:
# Pasamos las caras de training y testing a listas
data_training = []
for j in range(len(train_caras)):
  data_training.append(train_caras[j])

data_testing = []
for j in range(len(test_caras)):
  data_testing.append(test_caras[j])

# Convertimos las listas en matrices de tamaño (n, 900)
data_training = np.vstack(data_training)
#data_testing = np.vstack(data_testing)

In [ ]:
isomap = Isomap(n_components=mejor_n_components, n_neighbors=mejor_n_neighbors)
output_isomap_training = isomap.fit_transform(data_training)

#output_isomap_testing = isomap.transform(data_testing)

In [ ]:
df_train = pl.DataFrame(output_isomap_training)
df_train.columns = ['x' + str(i + 1) for i in range(df_train.shape[1])]
df_train = df_train.with_columns(pl.Series(train_nombres).alias("y"))

#df_test = pl.DataFrame(output_isomap_testing)
#df_test.columns = ['x' + str(i + 1) for i in range(df_test.shape[1])]
#df_test = df_test.with_columns(pl.Series(test_nombres).alias("y"))

In [ ]:
# Código para exportar el isomap en un .pkl
import pickle
path = '/content/.drive/My Drive/DMA/NuestrasCaras/isomap.pkl'

with open(path, 'wb') as f:
    pickle.dump(isomap, f)

### Construcción de variable respuesta

In [ ]:
campos = []
for i in range(mejor_n_components):
  campos.append(f"x{i+1}")

## 2  Entrenamiento del modelo

### 2.1  Inicializacion de la neural network



*   Es valido cambiar la *semilla_red* para arrancar el entrenamiento con distintas rectas iniciales


In [ ]:
# defino la red multiperceptron
carpeta = "/content/buckets/b1/nn_prueba/"  # cambiar con cada corrida
semilla_red = 547389  # define las rectas iniciales

# una sola capa oculta de 2 neuronas  [2]
# la capa oculta y la final tienen ambas logsig de activacion
mp = multiperceptron()
mp.inicializar(
    df=df_train, campos=campos, clase="y",  # especificaion del dataset
    hidden_layers_sizes=[30,20],  # no va la capa final, solo hidden layers
    layers_func=['logsig','logsig','logsig'], # funciones de activacion de cada capa
    semilla=semilla_red,
    carpeta = carpeta
    )

### 2.2 Entrenamiento de la neural network = backpropagation

Aqui se hace el trabajo pesado de entrenar la red neuronal

Es necesario experimentar con


*   learning_rate
*   lr_momento
*   epoch_limit  y MSE_umbral



In [ ]:
# entreno la neural netowork con BackPropagation

# el entrenamiento
(epoch, MSE, train_error_rate) = mp.entrenar(
    epoch_limit=2500,
    MSE_umbral=0.0001,
    learning_rate=0.2,
    lr_momento=0.2,
    save_frequency=100,
    retomar=True)


#### Visualizacion de los resultados de la salida del entrenamiento de la red

In [ ]:
# las metricas basica de la red
print("epoch :", epoch)
print("MSE :", MSE)
print("train_error_rate :", train_error_rate)

In [ ]:
# la primera hidden layer
print("W :", mp.red["layer"][0]["W"])
print()
print("w0 :", mp.red["layer"][0]["w0"])

## 3  Prediccion en los datos de Testing


Se muestran los datos de testing, que son distintos a los de training

### 3.1 Prediccion en caliente

In [ ]:
# aplico la red entrenada al dataset de testing

(y_hat,y_raw, test_error_rate) = mp.predecir(df_new=df_test, campos=campos, clase='y')

#### Visualizacion del error en testing

In [ ]:
print("error_rate (train, test): ",  train_error_rate, test_error_rate)

#### Visualizacion de la prediccion en testing

In [ ]:
tb_salida_test = pl.DataFrame( {"clase":df_test["y"], "pred":y_hat, "y_raw":y_raw })
tb_salida_test

#### Cálculo de errores por persona

In [ ]:
# Paso 1: crear una columna booleana que indique si acertó o no
df = tb_salida_test.with_columns([
    (pl.col("clase") != pl.col("pred")).alias("error")
])

# Paso 2: agrupar por clase real y calcular tasa de error por clase
errores_por_clase = (
    df.group_by("clase")
    .agg([
        pl.len().alias("total"),
        pl.col("error").sum().alias("errores")
    ])
    .with_columns([
        (pl.col("errores") / pl.col("total")).alias("error_rate")
    ])
    .sort("error_rate", descending=True)
)
pl.Config.set_tbl_rows(20)
print(errores_por_clase)

In [ ]:
def analisis_fotos(fotos_reales, prediccion, fotos_error = True):
  for i in range(len(prediccion)):
    if ((fotos_error and prediccion['error'][i]) or (not(fotos_error) and not(prediccion['error'][i]))):
      print(f"La foto era de {prediccion['clase'][i]} y la red dijo que era de {prediccion['pred'][i]}")
      plt.figure(figsize=(1, 1))
      plt.imshow(test_caras[i].reshape(50, 50), cmap='gray')
      plt.axis('off')
      plt.show()

In [ ]:
analisis_fotos(data_testing, df, True)
